## Maestría Ciencias de la Computación
### Curso: Visión Artificial
#### Integrantes:
* Abel Edmundo Borit Guitton
* Luis Alberto Borit Guitton
* Betzy Jacqueline Yarin Ramıirez

#Template Matching
Se ha realizado este proyecto con cuda y C++
1. Subir la Imagen original (Google Colab)
2. Para cambiar la ruta de la imagen recién cargada ir a la línea 56
3. Subir el Template. Para cambiar la ruta de la imagen recién cargada ir a la línea 57
4. Para ajustar ir a la línea 95 (umbral)
5. Se generará una nueva imagen y la coincidencia estará encerrada en un marco blanco con el nombre @Resultado_001.png

In [1]:
!pip install nvcc4jupyter -q
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpi3qsdgua".


In [2]:
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image.h -q
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h -O stb_image_write.h -q

In [7]:
%%cuda

#include <iostream>
#include <cmath>
#include <vector>
#include <algorithm>
#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "/content/stb_image.h"
#include "/content/stb_image_write.h"

// Función para calcular la suma de los cuadrados de las diferencias entre dos regiones de imágenes
__device__ float squaredDifference(const unsigned char* image, const unsigned char* templateImage, int x, int y, int templateWidth, int templateHeight, int imageWidth, int channels) {
    float sum = 0.0f;
    for (int dy = 0; dy < templateHeight; ++dy) {
        for (int dx = 0; dx < templateWidth; ++dx) {
            for (int c = 0; c < channels; ++c) {
                int imageIndex = ((y + dy) * imageWidth + (x + dx)) * channels + c;
                int templateIndex = (dy * templateWidth + dx) * channels + c;
                float diff = image[imageIndex] - templateImage[templateIndex];
                sum += diff * diff;
            }
        }
    }
    return sum;
}

// Función kernel para aplicar Template Matching y dibujar rectángulos
__global__ void templateMatchingAndDrawRectangles(const unsigned char* image, const unsigned char* templateImage, unsigned char* result, int imageWidth, int imageHeight, int templateWidth, int templateHeight, int channels, float threshold) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < imageWidth - templateWidth && y < imageHeight - templateHeight) {
        // Calcular la suma de los cuadrados de las diferencias para esta región
        float diff = squaredDifference(image, templateImage, x, y, templateWidth, templateHeight, imageWidth, channels);
        // Si la diferencia es menor que el umbral, considerar que hay una coincidencia
        if (diff < threshold) {
            // Dibujar rectángulo alrededor de la región coincidente
            for (int dy = 0; dy < templateHeight; ++dy) {
                for (int dx = 0; dx < templateWidth; ++dx) {
                    for (int c = 0; c < channels; ++c) {
                        int resultIndex = ((y + dy) * imageWidth + (x + dx)) * channels + c;
                        // Marcar el píxel como blanco si está en el borde del template
                        if (dy == 0 || dy == templateHeight - 1 || dx == 0 || dx == templateWidth - 1) {
                            result[resultIndex] = 255; // Canal rojo
                        }
                    }
                }
            }
        }
    }
}

int main() {
    // Cargar la imagen y el template desde archivos usando la función stbi_load
    const char* imageFilename = "/content/imagen.png";
    const char* templateFilename = "/content/template.png";
    int imageWidth, imageHeight, imageChannels;
    unsigned char *image = stbi_load(imageFilename, &imageWidth, &imageHeight, &imageChannels, 0);
    if (!image) {
        std::cerr << "Error al cargar la imagen" << std::endl;
        return 1;
    }

    int templateWidth, templateHeight, templateChannels;
    unsigned char *templateImage = stbi_load(templateFilename, &templateWidth, &templateHeight, &templateChannels, 0);
    if (!templateImage) {
        std::cerr << "Error al cargar el template" << std::endl;
        return 1;
    }

    // Definir el tamaño de la imagen resultante
    size_t resultSize = imageWidth * imageHeight * imageChannels;
    unsigned char *result = new unsigned char[resultSize];

    // Definir el tamaño de la imagen y del template
    size_t imageSize = imageWidth * imageHeight * imageChannels;
    size_t templateSize = templateWidth * templateHeight * templateChannels;

    // Reservar memoria en la GPU
    unsigned char *d_image, *d_template, *d_result;
    cudaMalloc((void **)&d_image, imageSize);
    cudaMalloc((void **)&d_template, templateSize);
    cudaMalloc((void **)&d_result, resultSize);

    // Copiar la imagen y el template desde la memoria de la CPU a la memoria de la GPU
    cudaMemcpy(d_image, image, imageSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_template, templateImage, templateSize, cudaMemcpyHostToDevice);

    // Definir el tamaño de bloque y el tamaño de la cuadrícula
    dim3 blockSize(32, 32);
    dim3 gridSize((imageWidth + blockSize.x - 1) / blockSize.x, (imageHeight + blockSize.y - 1) / blockSize.y);

    // Definir el umbral para la coincidencia
    float threshold = 10000.0f;

    // Lanzar el kernel para aplicar Template Matching y dibujar rectángulos
    templateMatchingAndDrawRectangles<<<gridSize, blockSize>>>(d_image, d_template, d_result, imageWidth, imageHeight, templateWidth, templateHeight, imageChannels, threshold);

    // Copiar la imagen resultante desde la memoria de la GPU a la memoria de la CPU
    cudaMemcpy(result, d_result, resultSize, cudaMemcpyDeviceToHost);

    // Combinar la imagen principal y el resultado del rectángulo
    for (int i = 0; i < imageSize; ++i) {
        if (result[i] == 255) {
            // Usar el canal rojo del resultado para el rectángulo
            image[i] = result[i];
        }
    }

    // Guardar la imagen resultante usando stbi_write
    stbi_write_png("/content/@Resultado_001.png", imageWidth, imageHeight, imageChannels, image, imageWidth * imageChannels);

    std::cout << "Se ha generado la imagen resultante con rectángulos superpuestos en /content/@Resultado_001.png" << std::endl;

    // Liberar memoria en la GPU
    cudaFree(d_image);
    cudaFree(d_template);
    cudaFree(d_result);

    // Liberar la memoria de las imágenes cargadas
    stbi_image_free(image);
    stbi_image_free(templateImage);
    delete[] result;

    return 0;
}


Se ha generado la imagen resultante con rectángulos superpuestos en /content/@Resultado_001.png

